In [1]:
# Run prediction and genertae pixelwise annotation for every pixels in the image using fully coonvolutional neural net
# Output saved as label images, and label image overlay on the original image
# 1) Make sure you you have trained model in logs_dir (See Train.py for creating trained model)
# 2) Set the Image_Dir to the folder where the input image for prediction are located
# 3) Set number of classes number in NUM_CLASSES
# 4) Set Pred_Dir the folder where you want the output annotated images to be save
# 5) Run script
#--------------------------------------------------------------------------------------------------------------------
import tensorflow as tf
import numpy as np
import scipy.misc as misc
import sys
from tensorflow.python.framework import ops
from tensorflow.python.ops import gen_nn_ops

In [2]:

import BuildNetVgg16
import TensorflowUtils
import os
import Data_Reader
import OverrlayLabelOnImage as Overlay
import CheckVGG16Model
logs_dir= "logs/"# "path to logs directory where trained model and information will be stored"
Image_Dir="Data_Zoo/test/"# Test image folder
w=0.6# weight of overlay on image
Pred_Dir="Output_Prediction/" # Library where the output prediction will be written
model_path="Model_Zoo/vgg16.npy"# "Path to pretrained vgg16 model for encoder"
NameEnd="" # Add this string to the ending of the file name optional
NUM_CLASSES = 2 # Number of classes
#-------------------------------------------------------------------------------------------------------------------------
CheckVGG16Model.CheckVGG16(model_path)# Check if pretrained vgg16 model avialable and if not try to download it


In [3]:
################################################################################################################################################################################

      # .........................Placeholders for input image and labels........................................................................
keep_prob = tf.placeholder(tf.float32, name="keep_probabilty")  # Dropout probability
image = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="input_image")  # Input image batch first dimension image number second dimension width third dimension height 4 dimension RGB


In [4]:

    # -------------------------Build Net----------------------------------------------------------------------------------------------
Net = BuildNetVgg16.BUILD_NET_VGG16(vgg16_npy_path=model_path)  # Create class instance for the net


npy file loaded


In [5]:

Net.build(image, NUM_CLASSES, keep_prob)  # Build net and load intial weights (weights before training)


build model started


Instructions for updating:
Use the `axis` argument instead


FCN model built


In [6]:

    # -------------------------Data reader for validation/testing images-----------------------------------------------------------------------------------------------------------------------------
ValidReader = Data_Reader.Data_Reader(Image_Dir,  BatchSize=1)


In [7]:

    #-------------------------Load Trained model if you dont have trained model see: Train.py-----------------------------------------------------------------------------------------------------------------------------

sess = tf.InteractiveSession() #Start Tensorflow session

print("Setting up Saver...")
saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())
ckpt = tf.train.get_checkpoint_state(logs_dir)
if ckpt and ckpt.model_checkpoint_path: # if train model exist restore it
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("Model restored...")
else:
    print("ERROR NO TRAINED MODEL IN: "+ckpt.model_checkpoint_path+" See Train.py for creating train network ")
    sys.exit()


Setting up Saver...


INFO:tensorflow:Restoring parameters from logs/model.ckpt-100000


Model restored...


## --My Tests start--

In [ ]:
import  imageio
Images = imageio.imread('Data_Zoo/test/acl2.jpg')
Images.shape
Images = Images[:,:,0:3]
Images.shape
import matplotlib.pyplot as plt
plt.imshow(Images)
plt.show()
Images = np.reshape(Images,[1,600,900,3])

In [15]:
LabelPred = Net.Prob.eval( feed_dict={image: Images, keep_prob: 1.0})

In [16]:
LabelPred.shape

(1, 600, 900, 2)

In [17]:
pre1 = LabelPred[0,:,:,0]
pre2 = LabelPred[0,:,:,1]

In [18]:
pre1.shape
pre2.shape

(600, 900)

In [19]:
plt.figure(1)
plt.imshow(pre1)
plt.figure(2)
plt.imshow(pre2)
plt.show()

In [20]:
finalpre = np.argmax(LabelPred,axis=3)

In [21]:
finalpre.shape

(1, 600, 900)

In [22]:
finalpre = finalpre[0,:,:]
plt.figure(3)
plt.imshow(finalpre)
plt.show()

In [23]:
# start draw saliency map
#get input
inpt = Net.bgr
#get output
outp = Net.Prob

In [24]:
background, object = tf.split(outp,num_or_size_splits=2,axis=3)

# Start saliency map

In [25]:
#get the max of output
#maxmum = tf.reduce_max(outp,axis=1,keep_dims=True)
#print(maxmum.shape)
# get the saliency map
saliency = tf.gradients(object,inpt)
print(len(saliency))
saliencymap = saliency[0]
saliencymap = sess.run(saliencymap, feed_dict={image: Images, keep_prob: 1.0})
saliencymap.shape


1


(1, 600, 900, 3)

In [26]:
saliencymap = np.reshape(saliencymap,[600,900,3])

# convert BGR to RGB
saliencymap_rgb = np.zeros([600,900,3])
saliencymap_rgb[:,:,0] = saliencymap[:,:,2]
saliencymap_rgb[:,:,1] = saliencymap[:,:,1]
saliencymap_rgb[:,:,2] = saliencymap[:,:,0]
saliencymap_abs = np.abs(saliencymap_rgb).max(axis=-1)

plt.figure()
plt.imshow(saliencymap_abs)
plt.show()


possaliency = (np.maximum(0, saliencymap_rgb) / saliencymap_rgb.max())
plt.figure()
plt.imshow(possaliency)
plt.show()


## --My tests end--

# start guided back propagation

In [3]:
import  imageio
Images = imageio.imread('Data_Zoo/test/acl2.jpg')
Images.shape
Images = Images[:,:,0:3]
Images.shape
import matplotlib.pyplot as plt
plt.imshow(Images)
plt.show()
Images = np.reshape(Images,[1,600,900,3])

In [4]:
@tf.RegisterGradient("GuidedRelu")
def _GuidedReluGrad(op, grad):
    return tf.where(0. < grad, gen_nn_ops._relu_grad(grad, op.outputs[0]), tf.zeros_like(grad))
#修改为grad和op都大于零


In [5]:


g = tf.get_default_graph()
with g.gradient_override_map({"Relu": "GuidedRelu"}):
    #load the net
    # .........................Placeholders for input image and labels........................................................................
    keep_prob = tf.placeholder(tf.float32, name="keep_probabilty")  # Dropout probability
    image = tf.placeholder(tf.float32, shape=[None, None, None, 3], name="input_image")  # Input image batch first dimension image number second dimension width third dimension height 4 dimension RGB
    # -------------------------Build Net----------------------------------------------------------------------------------------------
    Net = BuildNetVgg16.BUILD_NET_VGG16(vgg16_npy_path=model_path)  # Create class instance for the net
    Net.build(image, NUM_CLASSES, keep_prob)    
    
    #get input
    inpt = Net.bgr
    #get output
    outp = Net.Prob
    background, object = tf.split(outp,num_or_size_splits=2,axis=3)
    
    guided_back = tf.gradients(background,inpt)
    print(len(guided_back))
    guided_back_map = guided_back[0]
    sess1 = tf.Session()
    sess1.run(tf.global_variables_initializer())
    guided_back_map = sess1.run(guided_back_map, feed_dict={image: Images, keep_prob: 1.0})
        
print(guided_back_map.shape)


In [6]:
guided_back_map = np.reshape(guided_back_map,[600,900,3])

# convert BGR to RGB
guided_back_map_rgb = np.zeros([600,900,3])
guided_back_map_rgb[:,:,0] = guided_back_map[:,:,2]
guided_back_map_rgb[:,:,1] = guided_back_map[:,:,1]
guided_back_map_rgb[:,:,2] = guided_back_map[:,:,0]
guided_back_map_abs = np.abs(guided_back_map_rgb).max(axis=-1)
plt.figure()
plt.imshow(guided_back_map_abs)
plt.show()
pos_guided_back = (np.maximum(0, guided_back_map_rgb) / guided_back_map_rgb.max())
plt.figure()
plt.imshow(pos_guided_back)
plt.show()


# __End__

In [23]:

#--------------------Create output directories for predicted label, one folder for each granulairy of label prediciton---------------------------------------------------------------------------------------------------------------------------------------------

if not os.path.exists(Pred_Dir): os.makedirs(Pred_Dir)
if not os.path.exists(Pred_Dir+"/OverLay"): os.makedirs(Pred_Dir+"/OverLay")
if not os.path.exists(Pred_Dir + "/Label"): os.makedirs(Pred_Dir + "/Label")

    
print("Running Predictions:")
print("Saving output to:" + Pred_Dir)


Running Predictions:
Saving output to:Output_Prediction/


In [24]:
#----------------------Go over all images and predict semantic segmentation in various of classes-------------------------------------------------------------
fim = 0
print("Start Predicting " + str(ValidReader.NumFiles) + " images")
while (ValidReader.itr < ValidReader.NumFiles):
    print(str(fim * 100.0 / ValidReader.NumFiles) + "%")
    fim += 1
        # ..................................Load image.......................................................................................
    FileName=ValidReader.OrderedFiles[ValidReader.itr] #Get input image name
    Images = ValidReader.ReadNextBatchClean()  # load testing image

        # Predict annotation using net
    LabelPred = sess.run(Net.Pred, feed_dict={image: Images, keep_prob: 1.0})
    

Start Predicting 5 images
0.0%


20.0%


40.0%


60.0%


80.0%


In [11]:
print(LabelPred.shape)
type(LabelPred)

(1, 600, 900)


numpy.ndarray

In [17]:
import matplotlib.pyplot as plt
pred = np.reshape(LabelPred,[600,900])
plt.imshow(pred)
plt.show()

The history saving thread hit an unexpected error (OperationalError('unable to open database file',)).History will not be written to the database.


In [14]:
pred[150:450,150:450]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [ ]:

             #------------------------Save predicted labels overlay on images---------------------------------------------------------------------------------------------
    misc.imsave(Pred_Dir + "/OverLay/"+ FileName+NameEnd  , Overlay.OverLayLabelOnImage(Images[0],LabelPred[0], w)) #Overlay label on image
    misc.imsave(Pred_Dir + "/Label/" + FileName[:-4] + ".png" + NameEnd, LabelPred[0].astype(np.uint8))


Start Predicting 5 images
0.0%


20.0%


40.0%


60.0%


80.0%


In [ ]:
##################################################################################################################################################
print("Finished")